# Библиотеки

In [1]:
import pandas as pd
import numpy as np
import neptune

from NN import simple_torchpl
from load_data import load
from pl_framework import nn_training
from cv import get_indices

import torch

In [2]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

In [3]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzI0ZTMzMDQtZTFmZi00ZjgxLWI4NGQtZGJiOWQyNDJiYjE5In0="
os.environ['NEPTUNE_PROJECT']="iliaavilov/SIBUR"
os.environ['NEPTUNE_NOTEBOOK_ID']="6eddcb21-b0fa-4073-a5c6-f26572800a8b"
os.environ['NEPTUNE_NOTEBOOK_PATH']="Desktop/Projects/sibur_2020/modeling.ipynb"

# Константы

In [4]:
data_path = 'data/'
random_state = 54321
pd.options.display.max_rows = 999
features = ['A_rate', 'A_CH4', 'A_C2H6', 'A_C3H8', 'A_iC4H10', 'A_nC4H10',
            'A_iC5H12', 'A_nC5H12', 'A_C6H14', 'B_rate']

# Загрузка данных

In [5]:
train_features, train_targets, test_features = load(data_path)

In [6]:
train_features = train_features[(train_features['timestamp'] > pd.to_datetime('2020-02-15 00:00:00'))].reset_index(drop = True)
train_targets = train_targets[(train_targets['timestamp'] > pd.to_datetime('2020-02-15 00:00:00'))].reset_index(drop = True)

In [7]:
cv = get_indices(train_targets, [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                                 (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
#                                 (pd.to_datetime('2020-03-31 00:00:00'), pd.to_datetime('2020-04-15 00:00:00')),
                                 (pd.to_datetime('2020-04-15 00:00:00'), pd.to_datetime('2020-04-30 00:00:00'))
                                ]
                )

In [8]:
train_targets = train_targets.drop('timestamp', axis = 'columns')

# Предсказания для теста

In [9]:
params_model = {'n_in': 21, 'n_h_1': 1216, 'n_h_2': 1075, 'batch_size': 2793, 'p_1': 0.4282193995049264, 'p_2': 0.6010574543964716, 'p_3': 0.7834989659285684, 'activation1': 'Hardshrink', 'activation2': 'Hardtanh', 'optimizer': 'AdamW', 'lr': 0.00037390697257767716, 'weight_decay': 0.06340977765938485, 'n_out': 4, 'mean_best_iter': 99}

In [10]:
def prepare_features(n_in, train_features):
    train_features['grouper'] = 1
    final_features = pd.DataFrame()
    for n in range(n_in, train_features.shape[0]):
        n_in_previous = train_features.iloc[(n - n_in):(n + 1), :]
        previous_pivoted = pd.pivot_table(n_in_previous, index='grouper', columns='timestamp')
        previous_pivoted.columns = previous_pivoted.columns.get_level_values(0) + \
                                   '_' + \
                                   pd.Series(list(range(0, n_in + 1)) * 10).astype(str)
        final_features = final_features.append(previous_pivoted.transpose()[1])
    final_features.index = train_features.iloc[n_in:, :].index

    return final_features

all_features = train_features.append(test_features, ignore_index = True)

n_in = params_model['n_in']
batch_size = params_model['batch_size']
params_model.pop('batch_size')
mean_best_iter = params_model['mean_best_iter']
params_model.pop('mean_best_iter')

all_features = prepare_features(n_in, all_features)
X_train = all_features.loc[:(train_features.shape[0]-1), :]
X_test = all_features.loc[(train_features.shape[0]):, :]

X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)

y_train  = train_targets
y_train = y_train[n_in:].reset_index(drop = True)

In [11]:
fold = cv[0]
fold[0] = fold[0][n_in:]
fold[1] = fold[0]

In [12]:
fold

[Int64Index([ 21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
             ...
             709, 710, 711, 712, 713, 714, 715, 716, 717, 718],
            dtype='int64', length=698),
 Int64Index([ 21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
             ...
             709, 710, 711, 712, 713, 714, 715, 716, 717, 718],
            dtype='int64', length=698)]

In [13]:
my_boiii = nn_training(simple_torchpl, X_train, y_train)

my_boiii.train(min_epochs=mean_best_iter,
               max_epochs=mean_best_iter,
               model_params=params_model,
               batch_size=batch_size,
               fold=fold,
               val_fold=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | activ1       | Hardshrink | 0     
1 | activ2       | Hardtanh   | 0     
2 | dropout1     | Dropout    | 0     
3 | dropout2     | Dropout    | 0     
4 | dropout3     | Dropout    | 0     
5 | linear1      | Linear     | 268 K 
6 | linear2      | Linear     | 1 M   
7 | linear_final | Linear     | 4 K   
8 | loss         | MSELoss    | 0     
9 | loss_sec     | MAPE       | 0     


train_loss: 1.0138298273086548
val_loss: 0.8552809953689575
train_loss: 0.9332906603813171
val_loss: 0.7507258057594299
train_loss: 0.8604775071144104
val_loss: 0.7006155252456665
train_loss: 0.7947289943695068
val_loss: 0.6490312218666077
train_loss: 0.7431012988090515
val_loss: 0.6332719326019287
train_loss: 0.6902955174446106
val_loss: 0.600362241268158
train_loss: 0.6389673948287964
val_loss: 0.5726584792137146
train_loss: 0.6074444055557251
val_loss: 0.5451584458351135
train_loss: 0.5716885924339294
val_loss: 0.5162032842636108
train_loss: 0.5362913608551025
val_loss: 0.48928484320640564
train_loss: 0.5136035680770874
val_loss: 0.4616830050945282
train_loss: 0.4892360270023346
val_loss: 0.43724507093429565
train_loss: 0.4741166830062866
val_loss: 0.4221314787864685
train_loss: 0.46793022751808167
val_loss: 0.41838961839675903
train_loss: 0.45797544717788696
val_loss: 0.41350051760673523
train_loss: 0.44562938809394836
val_loss: 0.40994879603385925
train_loss: 0.43791133165359497
v

In [14]:
my_model1 = my_boiii.model
my_model1.eval()

simple_torchpl(
  (activ1): Hardshrink(0.5)
  (activ2): Hardtanh(min_val=-1.0, max_val=1.0)
  (dropout1): Dropout(p=0.4282193995049264, inplace=False)
  (dropout2): Dropout(p=0.6010574543964716, inplace=False)
  (dropout3): Dropout(p=0.7834989659285684, inplace=False)
  (linear1): Linear(in_features=220, out_features=1216, bias=True)
  (linear2): Linear(in_features=1216, out_features=1075, bias=True)
  (linear_final): Linear(in_features=1075, out_features=4, bias=True)
  (loss): MSELoss()
  (loss_sec): MAPE()
)

In [15]:
predictions = my_model1(torch.from_numpy(X_test.values).float()).detach().numpy()
submission = pd.DataFrame(predictions, columns = ['B_C2H6', 'B_C3H8', 'B_iC4H10', 'B_nC4H10'])
submission['timestamp'] = test_features['timestamp'].values
submission.to_csv('submission.csv', index = False)

In [16]:
#neptune.project.get_experiments('SIB-71')[0].log_metric('leaderboard_mape', 7.4781)

In [21]:
X_train

,A_C2H6_0,A_C2H6_1,A_C2H6_10,A_C2H6_11,A_C2H6_12,A_C2H6_13,A_C2H6_14,A_C2H6_15,A_C2H6_16,A_C2H6_17,...,B_rate_2,B_rate_20,B_rate_21,B_rate_3,B_rate_4,B_rate_5,B_rate_6,B_rate_7,B_rate_8,B_rate_9
0,4.287558,4.343639,4.422632,4.286216,4.046296,3.942095,3.819186,4.264613,4.314442,4.327101,...,71.281107,70.946832,70.746801,71.258247,71.320995,71.226476,71.233088,71.164417,71.182867,71.165650
1,4.343639,4.441232,4.286216,4.046296,3.942095,3.819186,4.264613,4.314442,4.327101,4.286105,...,71.258247,70.746801,70.847143,71.320995,71.226476,71.233088,71.164417,71.182867,71.165650,71.241606
2,4.441232,4.513587,4.046296,3.942095,3.819186,4.264613,4.314442,4.327101,4.286105,4.172174,...,71.320995,70.847143,70.918805,71.226476,71.233088,71.164417,71.182867,71.165650,71.241606,71.239426
3,4.513587,4.485880,3.942095,3.819186,4.264613,4.314442,4.327101,4.286105,4.172174,4.221487,...,71.226476,70.918805,70.984718,71.233088,71.164417,71.182867,71.165650,71.241606,71.239426,71.163024
4,4.485880,4.429670,3.819186,4.264613,4.314442,4.327101,4.286105,4.172174,4.221487,4.279978,...,71.233088,70.984718,70.828623,71.164417,71.182867,71.165650,71.241606,71.239426,71.163024,71.194365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3621,5.291033,5.281897,5.013553,5.031318,5.086050,5.083894,5.020284,5.004121,5.030980,5.063574,...,71.659833,73.998143,74.183355,71.878270,71.799849,71.799511,71.793446,71.762795,71.706573,71.511902
3622,5.281897,5.261989,5.031318,5.086050,5.083894,5.020284,5.004121,5.030980,5.063574,5.080800,...,71.878270,74.183355,74.213066,71.799849,71.799511,71.793446,71.762795,71.706573,71.511902,71.283077
3623,5.261989,5.245522,5.086050,5.083894,5.020284,5.004121,5.030980,5.063574,5.080800,5.088785,...,71.799849,74.213066,74.144367,71.799511,71.793446,71.762795,71.706573,71.511902,71.283077,71.466820
3624,5.245522,5.172751,5.083894,5.020284,5.004121,5.030980,5.063574,5.080800,5.088785,5.052961,...,71.799511,74.144367,74.320575,71.793446,71.762795,71.706573,71.511902,71.283077,71.466820,71.904219


In [23]:
def make_seqs(n, data):
    final_X = []

    def seq2seq(x):
        seqs_X.append(np.array(x).reshape(-1, 1))
        return (5)

    for col in data.columns:
        seqs_X = []
        data[col].rolling(n).apply(seq2seq)
        seqs_X = np.array(seqs_X)
        final_X.append(seqs_X)

    final_X = np.concatenate(final_X, axis=2)
    return (final_X)

In [24]:
final_X = make_seqs(2, X_train)

In [26]:
X_train

,A_C2H6_0,A_C2H6_1,A_C2H6_10,A_C2H6_11,A_C2H6_12,A_C2H6_13,A_C2H6_14,A_C2H6_15,A_C2H6_16,A_C2H6_17,...,B_rate_2,B_rate_20,B_rate_21,B_rate_3,B_rate_4,B_rate_5,B_rate_6,B_rate_7,B_rate_8,B_rate_9
0,4.287558,4.343639,4.422632,4.286216,4.046296,3.942095,3.819186,4.264613,4.314442,4.327101,...,71.281107,70.946832,70.746801,71.258247,71.320995,71.226476,71.233088,71.164417,71.182867,71.165650
1,4.343639,4.441232,4.286216,4.046296,3.942095,3.819186,4.264613,4.314442,4.327101,4.286105,...,71.258247,70.746801,70.847143,71.320995,71.226476,71.233088,71.164417,71.182867,71.165650,71.241606
2,4.441232,4.513587,4.046296,3.942095,3.819186,4.264613,4.314442,4.327101,4.286105,4.172174,...,71.320995,70.847143,70.918805,71.226476,71.233088,71.164417,71.182867,71.165650,71.241606,71.239426
3,4.513587,4.485880,3.942095,3.819186,4.264613,4.314442,4.327101,4.286105,4.172174,4.221487,...,71.226476,70.918805,70.984718,71.233088,71.164417,71.182867,71.165650,71.241606,71.239426,71.163024
4,4.485880,4.429670,3.819186,4.264613,4.314442,4.327101,4.286105,4.172174,4.221487,4.279978,...,71.233088,70.984718,70.828623,71.164417,71.182867,71.165650,71.241606,71.239426,71.163024,71.194365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3621,5.291033,5.281897,5.013553,5.031318,5.086050,5.083894,5.020284,5.004121,5.030980,5.063574,...,71.659833,73.998143,74.183355,71.878270,71.799849,71.799511,71.793446,71.762795,71.706573,71.511902
3622,5.281897,5.261989,5.031318,5.086050,5.083894,5.020284,5.004121,5.030980,5.063574,5.080800,...,71.878270,74.183355,74.213066,71.799849,71.799511,71.793446,71.762795,71.706573,71.511902,71.283077
3623,5.261989,5.245522,5.086050,5.083894,5.020284,5.004121,5.030980,5.063574,5.080800,5.088785,...,71.799849,74.213066,74.144367,71.799511,71.793446,71.762795,71.706573,71.511902,71.283077,71.466820
3624,5.245522,5.172751,5.083894,5.020284,5.004121,5.030980,5.063574,5.080800,5.088785,5.052961,...,71.799511,74.144367,74.320575,71.793446,71.762795,71.706573,71.511902,71.283077,71.466820,71.904219


In [32]:
final_X[1]

array([[ 4.34363867,  4.4412315 ,  4.28621633,  4.04629633,  3.942095  ,
         3.8191864 ,  4.264613  ,  4.31444167,  4.327101  ,  4.28610533,
         4.1721738 ,  4.221487  ,  4.5135874 ,  4.2799778 ,  4.17262467,
         4.48588   ,  4.42967033,  4.395051  ,  4.387076  ,  4.412086  ,
         4.43952967,  4.422632  , 39.24793   , 39.25664   , 39.53578333,
        39.46175   , 39.413375  , 39.341834  , 39.4847    , 39.46968667,
        39.41627333, 39.42505333, 39.579654  , 39.72221667, 39.28848   ,
        39.410254  , 38.91419   , 39.27022667, 39.22135   , 39.25452   ,
        39.416776  , 39.58354   , 39.70523667, 39.69353167,  5.70814533,
         5.7578015 ,  5.57140933,  5.535877  ,  5.5106125 ,  5.512624  ,
         5.56641233,  5.56745067,  5.55514583,  5.53904833,  5.4974518 ,
         5.439375  ,  5.7424246 ,  5.6458592 ,  6.01401933,  5.70420833,
         5.685801  ,  5.6781405 ,  5.6710018 ,  5.662814  ,  5.62603967,
         5.590495  ,  0.11823173,  0.11925805,  0.1